In [1]:
import sys
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
#import uproot
import pandas as pd
import statistics as st
import os
from numba import njit
import time
from numba.core.errors import NumbaDeprecationWarning, NumbaPendingDeprecationWarning
import warnings
from loguru import logger
from datetime import datetime

In [2]:
# warnings.simplefilter('ignore', category=NumbaDeprecationWarning)
# warnings.simplefilter('ignore', category=NumbaPendingDeprecationWarning)
# warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

# for arg in sys.argv:
#     run_num=str(arg).zfill(5)
#     # print(run_num)

# chan_enab = int(sys.argv[-1])
# run_start=str(sys.argv[1]).zfill(5)
# run_end=str(sys.argv[2]).zfill(5)
# run_num=str(sys.argv[3]).zfill(5)

run_num = '12034'
runs_folder = 'runs12034-12363/'
os.chdir('F:/LANL/')
datadir = 'sample_data/'
uniquefolder = 'debug_sample/'+runs_folder
SFNormFile = 'SF_Norm_files/'+runs_folder+run_num
statefileloc = 'F:\LANL\SF_Norm_files\TR_R_expected_avgs_stds_afterclip.csv'

# print(os.getcwd())

# os.chdir('F:/LANL/')
# datadir = 'D:/LANSCE_FP12_2023/data/' ## add directory of hard drive
# uniquefolder = "runs" + str(run_start) + "-" + str(run_end) +"/"
# SFNormFile = 'SF_Norm_files/'+uniquefolder+run_num

print('processing data: ' + uniquefolder + '/run' + run_num)

# statefileloc = 'F:\LANL\SF_Norm_files\TR_R_expected_avgs_stds_afterclip.csv'
# processedpulsefolder = '/processed_data/'+uniquefolder+'pulses_added_D/'
# processedasymfolder = '/processed_data/'+uniquefolder+'asym_D/'
# AddedPulseSavename = processedpulsefolder+run_num+'_pulsesadded_D'
# AsymSavename = processedasymfolder+run_num+'_asym_D'
# ONSavename = processedasymfolder+run_num+'_ON_D'
# OFFSavename = processedasymfolder+run_num+'_OFF_D'
# logger.add("F:/LANL/processed_data/" + uniquefolder + '0_ErrorLog_'+run_start+'_'+run_end+'_D.txt', delay = False)
# print('saving processed data to ' + AsymSavename)

## cannot handle all 24 detectors at once, memory issue... can look into np.empty and deleting variables if needed<br>
# chan_enab = np.array([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24]) ## all
chan_enab = np.array([0,1,2,3,4,5,6,7,8,9,10,11,24]) ## downstream
# chan_enab = np.array([12,13,14,15,16,17,18,19,20,21,22,23,24]) ## upstream

# if not os.path.exists(os.getcwd()+processedpulsefolder) or not os.path.exists(os.getcwd()+processedasymfolder):
#     # Create the directory
#     os.makedirs(os.getcwd()+processedpulsefolder)
#     os.makedirs(os.getcwd()+processedasymfolder)
#     print("Directory created successfully")
# else:
#     pass

start = time.time()
fullstart = time.time()

# read_data = np.array([])
# fileLength = np.array([])
read_data = []
fileLength = []

def open_file():
    for el in chan_enab:
        # f = open(datadir + folder + 'run' + run_num + "_ch" + str(el) + ".bin", 'rb')
        f = open(datadir+uniquefolder + 'run' + str(run_num) + "_ch" +str(el) + ".bin", 'rb')
        read_data.append(np.fromfile(file=f, dtype=np.uint16))
        f.close()
        fileLength.append(len(read_data[-1]))
    return read_data, fileLength

open_file()

fileLength = np.asarray(fileLength)
read_data = np.asarray(read_data) ## in detector's case, all are the same size samples, so can do read_data as np array

if chan_enab[-1] != 24:
    emessage = ('last channel is not 6Li detector')
    logger.error(run_num + emessage)
    raise Exception(emessage)

# print('saving processed data to ' + AsymSavename)
print("Channel is " + str(chan_enab))
end = time.time()
# print('file open time: ' + str(end-start))      
# print(read_data)

processing data: debug_sample/runs12034-12363//run12034
Channel is [ 0  1  2  3  4  5  6  7  8  9 10 11 24]


Store the big header for each channel in arrays

In [3]:
BoardID = []
recordLength = []
numSamples = []
eventCounter = []
decFactor = []
chanDec = []
postTrig = []
groupStart = []
groupEnd = []
timestamp= []
sizeFirstEvent = []
TTT = []

targetDict = {0: "La", 1: "Tb2O3", 2: "Yb2O3", 3: "Sm2O3", 4: "Er2O3", 5: "Ho2O3", 6: "other"}
foilDict = {0: "TBD", 1: "TBD", 2: "TBD", 3: "TBD", 4: "TBD", 5: "TBD", 6: "other"}

target=(read_data[0][5]&0x00F0)>>4
foil=read_data[0][5]&0x000F
targetFlag = read_data[0][5]>>8&1
foilFlag = read_data[0][5]>>9&1
spinFiltFlag = read_data[0][5]>>10&1
spinFlipFlag = read_data[0][5]>>11&1
shutterFlag = read_data[0][5]>>12&1
facilityTrigFlag = read_data[0][5]>>13&1

if targetFlag:
    target=targetDict[(read_data[0][5]&0x00F0)>>4]
else:
    target = "empty"
if foilFlag:
    foil=foilDict[read_data[0][5]&0x000F]
else:
    foil = "empty"
for i in range(0,len(chan_enab)):
    BoardID.append(read_data[i][9]>>8)
    recordLength.append(((read_data[i][9]&0x00FF)<<16)+read_data[i][8])
    numSamples.append(((read_data[i][11]&0x00FF)<<16)+read_data[i][10])
    eventCounter.append(read_data[i][6]+(read_data[i][7]<<16))
    BoardID.append(read_data[i][9]>>8)  
    decFactor.append(read_data[i][11]>>8)
    chanDec.append(read_data[i][13]>>8)
    postTrig.append(read_data[i][15]>>8)
    groupStart.append(((read_data[i][13]&0x00FF)<<16)+read_data[i][12])
    groupEnd.append(((read_data[i][15]&0x00FF)<<16)+read_data[i][14])
    timestamp.append(read_data[i][16]+(read_data[i][17]<<16)+(read_data[i][18]<<32)+(read_data[i][19]<<40))  
    sizeFirstEvent.append(read_data[i][0]+(read_data[i][1]<<16))
    TTT.append(read_data[i][2]+(read_data[i][3]<<16)+(read_data[i][4]<<32))
    
#     print("For channel " + str(chan_enab[i]) + ", BoardID is " + str(BoardID[i])
#           + "; record length is " + str(recordLength[i]) + "; num Samples is " 
#           + str(numSamples[i]) + "; event counter is " + str(eventCounter[i]) + "; dec factor is " + str(decFactor[i]) + "; chan dec is " 
#           + str(chanDec[i]) + "; postTrig is " + str(postTrig[i]) + "; group start is " + str(groupStart[i]) + "; group end is " + str(groupEnd[i])
#           + "; epoch time is " + str(timestamp[i]) +  "; first event size is " + str(sizeFirstEvent[i]) + "; and ETTT is " + str(TTT[i]) + "\n")

BoardID = np.asarray(BoardID) 
recordLength = np.asarray(recordLength)
numSamples = np.asarray(numSamples)
eventCounter = np.asarray(eventCounter)
decFactor = np.asarray(decFactor)
chanDec = np.asarray(chanDec)
postTrig = np.asarray(postTrig)
groupStart = np.asarray(groupStart)
groupEnd = np.asarray(groupEnd)
timestamp = np.asarray(timestamp)
sizeFirstEvent = np.asarray(sizeFirstEvent)
TTT = np.asarray(TTT)
print("Target is " + target)
# print("Foil is " + foil)
# print("Shutter is open: " + str(bool(shutterFlag)))
# print("Facility t0 is on: " + str(bool(facilityTrigFlag)))
# print("Spin flipper is on: " + str(bool(spinFlipFlag)))
# print("Spin filter is on: " + str(bool(spinFiltFlag)))
# print("Target is present: " + str(bool(targetFlag)))
# print("Foil is present: " + str(bool(foilFlag)))

Target is La


Determine the time axis for each channel

In [4]:
preTime = []
startTime = []
endTime = []
resolution = []
xs = [] 

for i in range(0,len(chan_enab)):
    preTime.append((100-postTrig[i])*recordLength[i]/100)
    startTime.append((-1*preTime[i]*16*decFactor[i] + groupStart[i]*16*decFactor[i]))
    endTime.append((-1*preTime[i]*16*decFactor[i] + groupEnd[i]*16*decFactor[i]))
    resolution.append(16*chanDec[i]*decFactor[i])
#     print("Pretime for channel", chan_enab[i],"is " + str(preTime[i]) + "; start time is " + str(startTime[i]) + "; end time is " + str(endTime[i]) 
#           + "; resolution is " + str(resolution[i]) + "ns")
    xs.append(np.arange(startTime[i],(numSamples[i])*resolution[i]+startTime[i], resolution[i]))

# np.asarray(preTime) 
# np.asarray(startTime) 
# np.asarray(endTime) 
# np.asarray(resolution)
xs = np.asarray(xs) ## can convert xs to np array here because all detectors same numsamples

In [5]:
print(resolution)

[512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512]


In [6]:
start=time.time()

@njit
def dataread(data, channels, fileLen, numSamps):
    numRuns = int((fileLen[0]-20-numSamps[0])/(numSamps[0]+6)+1)
    ys_arr = np.zeros((len(channels), numRuns,numSamps[0]), dtype=np.uint16)
    ETTT_arr = np.zeros((len(channels), numRuns), dtype=np.intc)
    eventcount_arr = np.zeros((len(channels), numRuns), dtype=np.intc)
    for i in range(0,len(channels)):
        eventCount = 0
        byteCounter = 0
            #byte counter is really 2bytecounter, lol
        while byteCounter < fileLen[i]:
            if byteCounter == 0:
                ETTT_arr[i]=TTT[i]
                #ETTT_arr[i].append(TTT[i])
                eventcount_arr[i]=(eventCounter[i])
                byteCounter = 20
            else:
                ETTT_arr[i]=(data[i][byteCounter]+(data[i][byteCounter+1]<<16)+(data[i][byteCounter+2]<<32))
                eventcount_arr[i]=(data[i][byteCounter+4]+(data[i][byteCounter+5]<<16))
                byteCounter += 6
            for j in range(0, numSamps[i]):
                #if j == 0:
                    #ys_arr[i].append([])
                #print(byteCounter)
                ys_arr[i][eventCount][j]=data[i][byteCounter]
                byteCounter += 1
            eventCount += 1
    return ys_arr, ETTT_arr, eventcount_arr

# start=time.time() 
# ys_arrHe, ETTT_arrHe, eventcount_arrHe  = dataread(read_data, [25], fileLength, numSamples) ##hardcoded channel 25 for He
ys_arr, ETTT_arr, eventcount_arr  = dataread(read_data, chan_enab, fileLength, numSamples) ##hardcoded channels for coils

end = time.time()
print('dataread from binary time: ' + str(end-start))

dataread from binary time: 1.7952797412872314


In [7]:
timeDif=[]
for i in range(0,len(chan_enab)):
    timeDif.append([])
    for j in range(len(ETTT_arr[i])-1):
        timeDif[i].append((ETTT_arr[i][j+1]-ETTT_arr[i][j])*8)
#     print("Min time difference for channel", chan_enab[i], "is", min(timeDif[i]), "ns")
#     print("Max time difference for channel", chan_enab[i], "is", max(timeDif[i]), "ns \n")
#print(timeDif)

Load in SF and He normalization information ##

In [8]:
try:
    df_SF = pd.read_hdf(SFNormFile + '.h5', key='df_0')
    df_HE = pd.read_hdf(SFNormFile + '.h5', key='df_1')
except Exception as e:
    logger.error(run_num + ' failed during SFNormFile load')
    logger.exception(e)

SF_Sort_arr = df_SF[['nicknames', 'transition_locations']].to_numpy().T
He_Norm_arr = df_HE[['pulse', 'norms']].to_numpy().T

NormFactor = 1000000  ## He integrals are huge, this normalizes all of those by a constant value for ease of use
HeNorms= (He_Norm_arr[1])/NormFactor

# print(SF_Sort_arr) 
# print(He_Norm_arr[1]/NormFactor)

In [9]:
# basesub and plotting ##

baseL = 0
baseR = int(((preTime[0]-groupStart[0])*0.70)/chanDec[0])  ##70% before the trigger
numRuns = int((fileLength[0]-20-numSamples[0])/(numSamples[0]+6)+1)
legend =  ['NaI', 'R']

start = time.time()

s = 20 ## pulse to look at 
t=s+1

#  dont know why this is so slow ##
def plotter(ys, xs, baseR, numpoints):
    tempys_basesub = np.zeros((len(ys), numRuns,numpoints[0]), dtype=float)
    for i in range((len(ys))):
        for pulse in range((len(eventcount_arr[0]))): ## all have 5000 pulses
            tempys_basesub[i][pulse]=np.subtract(ys[i][pulse], np.mean(ys[i][pulse][baseL:baseR]))
        for j in range(s, t): ## plot only interested pulses
            plt.plot(xs[i], tempys_basesub[i][j]) #label=legend[i]) #+str(sums[1][j])) ## sums[j] will not work for more than just TR   
            plt.axvline(xs[0][baseL], ls = '--')
            plt.axvline(xs[0][baseR], ls = '--')
            #plt.axvline(xs[0][int(((preTime[0]-groupStart[0])*0.70)/chanDec[0])], ls = '--', c ='m')
            plt.axvline(xs[0][baseR+5], ls = '--', c ='r') ## BaseR+5 line marks the beginning of the integral, until the end of samples.
#             plt.title('SF state transition' + transitions[p]) 
#             plt.xlabel("time from trigger (ns)")
#             plt.ylabel("ADC")
#             plt.legend()
            
# plotter(ys_arr[9:], xs[9:], baseR, numSamples) ##plot coils

ys_basesub = np.zeros((len(ys_arr), numRuns,numSamples[0]), dtype=np.float64)
# ys_basesub_norm = np.zeros((len(ys_arr), numRuns,numSamples[0]), dtype=np.float64)

@njit ## jit is faster for large # channels, slower for small # channels
def basesub(ys, baseRight, numpoints): 
    tempys_basesub = np.zeros((numRuns,numpoints[0]), dtype=np.float64)
    for pulse in range((len(eventcount_arr[0]))): ## all have 5000 pulses
        tempys_basesub[pulse]=np.subtract(ys[pulse], np.mean(ys[pulse][baseL:baseRight]))
    return tempys_basesub

@njit ## jit is faster for large # channels, slower for small # channels
def basesub_norm(ys, baseRight, numpoints): 
    tempys_basesub = np.zeros((numRuns,numpoints[0]), dtype=np.float64)
    for pulse in range((len(eventcount_arr[0]))): ## all have 5000 pulses
        tempys_basesub[pulse]=np.subtract(ys[pulse], np.mean(ys[pulse][baseL:baseRight]))
        tempys_basesub[pulse]=tempys_basesub[pulse]/HeNorms[pulse] 
    return tempys_basesub

# for peak finding algo., we don't want to use the normalization yet...
# for i in range(len(ys_basesub)): ## feeding y arrays into function 1 channel at  a time is faster than all at once
#     ys_basesub[i] = basesub(ys_arr[i], baseR, numSamples)
for i in range(len(ys_basesub)): ## if not using aligning/cutting later, ys should be normalized here
    ys_basesub[i] = basesub_norm(ys_arr[i], baseR, numSamples)

ys_basesub[-1] = ys_basesub[-1]*-1 ## invert 6Li to positive signal. Comment out if not using
# ys_basesub_norm[-1] = ys_basesub_norm[-1]*-1 ## invert 6Li to positive signal. Comment out if not using

end = time.time()
print('plotting and/or base subtraction time: ' + str(end-start))            

plotting and/or base subtraction time: 3.3978748321533203


In [10]:
%matplotlib qt
s = 20 ## pulse to look at 
t=s+1
    
leg =  ['NaI', 'R']
# plt.legend(leg)
# plt.title('One pulse') 
# plt.xlabel("time from trigger (ns)")
# plt.ylabel("ADC")
# ys_subset = ys_basesub[11][0][5000:9000].copy()
# xs_subset = xs[0][5000:9000].copy()
ys_subset = ys_basesub[1][0].copy()
xs_subset = xs[0].copy()

# peaks, _ = sp.signal.find_peaks(ys_subset, height=0)
# plt.plot(xs_subset, ys_subset)
# # plt.plot(ys_subset[peaks],peaks, "x")
# plt.show()

# # plt.axvline(xs[0][baseL], ls = '--')
# # plt.axvline(xs[0][baseR], ls = '--')
# plt.axvline(xs[0][7600], ls = '--', c ='g')
# plt.axvline(xs[0][8000], ls = '--', c ='g')
# plt.axvline(xs[0][1500], ls = '--', c ='g')

# plt.axvline(xs[1][intgrR], ls = '--', c ='g')
# plt.axvline(xs[2][HeintgrL], ls = '--', c ='r')
# plt.axvline(xs[2][HeintgrR], ls = '--', c ='r')

# plt.show()
# plotter(ys_basesub[10:], xs[10:], baseR, numSamples) ##plot coils


In [11]:
## a represents the bin location of peaks
a = [np.array([1,5,6,8,9,12,15,18,22,26,28]), np.array([1,5,6,8,9,12,15,18,22,26,28])*2, np.array([0,5,6,8,9,12,15,18,22,26,28])*4]
# print(a[2])
arange = [0,28]  ## what bins to look in (i.e. 0-1400 and 7600-9000)
# print(arange[0]-2)
# print(arange[1]+2)

# print((np.where((a[2]>=arange[0]) & (a[2]<=arange[1]))))
all_peaks = []
all_ranges = []
for i in range(len(a)): ## i represents every pulse?
    peakranges = []
#     temp_i=[] 
    temp_i = np.where((a[i]>=arange[0]) & (a[i]<=arange[1]))[0]
    peak_i = a[i][temp_i]
#     temp_i = np.array(temp_i[0].astype('int'))
    all_peaks.append(peak_i)
    print(peak_i)
    print()
#     if peak
    for j in (peak_i):  ## if clauses prevents summing of points before/after pulse begins/ends (i.e. peak in the beginning points)
        print(j)
        if j-2<arange[0]:
#             print(arange[0]-2)
            pass
        elif j+2>arange[1]:
            print()
            print(j)
            print()
            pass
        else:
            temp_range = [j-2, j+2]
    #         print(temp_range)
            peakranges.append(temp_range)
    all_ranges.append(peakranges)
print()
print(all_peaks)
print()
## all ranges represents the points (bins) in which to sum over.
print(all_ranges)
# print(all_ranges[0])
# print(all_ranges[0][0])
# print(all_ranges[0][0][0])
        

[ 1  5  6  8  9 12 15 18 22 26 28]

1
5
6
8
9
12
15
18
22
26
28

28

[ 2 10 12 16 18 24]

2
10
12
16
18
24
[ 0 20 24]

0
20
24

[array([ 1,  5,  6,  8,  9, 12, 15, 18, 22, 26, 28]), array([ 2, 10, 12, 16, 18, 24]), array([ 0, 20, 24])]

[[[3, 7], [4, 8], [6, 10], [7, 11], [10, 14], [13, 17], [16, 20], [20, 24], [24, 28]], [[0, 4], [8, 12], [10, 14], [14, 18], [16, 20], [22, 26]], [[18, 22], [22, 26]]]


In [12]:
print(len(all_peaks))
print(len(all_ranges[0][0]))
print(all_peaks[0])
# print(all_ranges)
print(all_ranges[0])



3
2
[ 1  5  6  8  9 12 15 18 22 26 28]
[[3, 7], [4, 8], [6, 10], [7, 11], [10, 14], [13, 17], [16, 20], [20, 24], [24, 28]]


In [13]:
print(len(ys_basesub[0][100][2:-2]))

8996


### find peaks, integrate

In [14]:
## I think this has been confirmed to work using the singular finding method done earlier (below in troubleshooting).
# @njit
def find_peaks(ys, pthresh, peakrange): ## ys here is for one channel!
#     pulse_arr = [[],[]] ## peaks index, pulse sum ranges
    all_peaks = []
    sum_ranges = []  ## all sum ranges  (i.e. for peak at bin 7, sum from bins 5-9. Save this for every peak)
    for p in range(len(ys)):  ## find peaks for every pulse p in one detector channel
        p_sum_ranges = []  ## one array of all sum ranges for a pulse p
        peaks, _ = sp.signal.find_peaks(ys[p], threshold=pthresh)#, height  = [1,1000])
        temp_i = np.array(np.where((peaks>=peakrange[0]) & (peaks<=peakrange[1]))[0], copy=True)
        peaks_i = peaks[temp_i]
#         all_peaks.append(peaks_i)
        for peak in peaks_i:
            if peak-2<peakrange[0]: ## plus and minus 2 x points (512 ns each) was determined from peak widths below
#                 print('peak in lower bound at pulse ' +str(p) +', peak '+str(peak))
                pass
            elif peak+2>peakrange[1]:
#                 print('pulse ' +str(p))
#                 print('peak in upper bound at pulse ' +str(p) +', peak '+str(peak))
                pass
            else:
                pm_range = [peak-2, peak+2]  #plus minus 2 bin range around found peak
                p_sum_ranges.append(pm_range)
        sum_ranges.append(p_sum_ranges)
#     return sum_ranges #, all_peaks ## don't necessarily need all_peaks array, keep it for troubleshoot
    return sum_ranges #, all_peaks ## don't necessarily need all_peaks array, keep it for troubleshoot
#     return all_peaks #, all_peaks ## don't necessarily need all_peaks array, keep it for troubleshoot


# peakthresh = 2 ## threshold for peaks is estimated
# peak_beg = 1400
# peak_end = 7600
# peak_range_beg = [0, peak_beg] ## hardcoded for 0-1400 and 7600-9000
# peak_range_end = [peak_end, 9000]

# peaks_test_beg, sum_ranges_test_beg = find_peaks(ys_basesub[0], peakthresh, peak_range_beg)
# peaks_test_end, sum_ranges_test_end = find_peaks(ys_basesub[0], peakthresh, peak_range_end)


In [15]:
## this is slow, about 40 seconds each. maybe reduce to just peaks, then do the integration with +/- 2 bins?
## if only doing all_peaks (take out all sum_ranges) reduces ~40s -> 22s
## if doing sum_ranges, still takes ~40s

start = time.time()
peakthresh = 2 ## threshold for peaks is estimated
peak_beg = 1399
peak_end = 7599
peak_range_beg = [0, peak_beg] ## hardcoded for 0-1400 and 7600-9000
peak_range_end = [peak_end, 8999]

sum_ranges_beg = []
sum_ranges_end = []

for i in range(len(ys_basesub)):
#     sum_ranges_beg, peaks_beg = find_peaks(ys_basesub[i], peakthresh, peak_range_beg)
#     sum_ranges_end, peaks_end = find_peaks(ys_basesub[i], peakthresh, peak_range_end)
    sum_ranges_beg.append(find_peaks(ys_basesub[i], peakthresh, peak_range_beg))
    sum_ranges_end.append(find_peaks(ys_basesub[i], peakthresh, peak_range_end))
#     peaks_beg  = find_peaks(ys_basesub[i], peakthresh, peak_range_beg)
#     peaks_end  = find_peaks(ys_basesub[i], peakthresh, peak_range_end)

end = time.time()
print(end-start)

37.60266709327698


In [16]:
# peak lengths are defined per pulse after being found; can maybe using numpy after that??
# split into 2 functions to use JIT?

## I think this has been confirmed to work using the singular finding method done earlier (below in troubleshooting).
# @njit
def find_peaks_np(ys, pthresh, peakrange): ## ys here is for one channel!
#     pulse_arr = [[],[]] ## peaks index, pulse sum ranges
    all_peaks = []
    sum_ranges = []  ## all sum ranges  (i.e. for peak at bin 7, sum from bins 5-9. Save this for every peak)
#     sum_ranges = np.zeros(len(ys))
    for p in range(len(ys)):  ## find peaks for every pulse p in one detector channel
#         print(p)
        peaks, _ = sp.signal.find_peaks(ys[p], threshold=pthresh)#, height  = [1,1000])
        temp_i = np.array(np.where((peaks>=peakrange[0]) & (peaks<=peakrange[1]))[0], copy=True)
        peaks_i = peaks[temp_i]
        p_sum_ranges = np.zeros((len(peaks_i), 2), dtype = int)
        if peaks_i[0]-2<peakrange[0]: ## plus and minus 2 x points (512 ns each) was determined from peak widths below
#                 print('peak in lower bound at pulse ' +str(p) +', peak '+str(peak))
            peaks_i = peaks_i[1:].copy()
            if peaks_i[-1]+2>peakrange[1]: ## this is in case both peak at beginning at end. Couldn't find a more elegant solution
                peaks_i = peaks_i[:-1].copy()
        elif peaks_i[-1]+2>peakrange[1]:
            peaks_i = peaks_i[:-1].copy()
#                 print('pulse ' +str(p))
#                 print('peak in upper bound at pulse ' +str(p) +', peak '+str(peak))
        else:
            peaks_i = peaks[temp_i]
        p_sum_ranges = np.zeros((len(peaks_i), 2), dtype = int)
#         print(peaks_i)
#         print(len(p_sum_ranges))## one array of all sum ranges for a pulse p
#         print(len(p_sum_ranges[0]))## one array of all sum ranges for a pulse p
#         print(len(p_sum_ranges[0][0]))## one array of all sum ranges for a pulse p
#         all_peaks.append(peaks_i)
        for peak in range(len(peaks_i)):
#             if peaks_i[peak]-2<peakrange[0]: ## plus and minus 2 x points (512 ns each) was determined from peak widths below
# #                 print('peak in lower bound at pulse ' +str(p) +', peak '+str(peak))
#                 pass
#             elif peaks_i[peak]+2>peakrange[1]:
# #                 print('pulse ' +str(p))
# #                 print('peak in upper bound at pulse ' +str(p) +', peak '+str(peak))
#                 pass
#             else:
#                 print(peak)
#                 print(peaks_i[peak])
#                 pm_range = np.array([peaks_i[peak]-2, peaks_i[peak]+2])  #plus minus 2 bin range around found peak
            p_sum_ranges[peak][0] = peaks_i[peak]-2
            p_sum_ranges[peak][1] = peaks_i[peak]+2#plus minus 2 bin range around found peak
#                 print(pm_range)  #plus minus 2 bin range around found peak
#                 print(len(pm_range)) 
#                 p_sum_ranges[peak] = pm_range
        sum_ranges.append(p_sum_ranges)
#     return sum_ranges #, all_peaks ## don't necessarily need all_peaks array, keep it for troubleshoot
    return sum_ranges #, all_peaks ## don't necessarily need all_peaks array, keep it for troubleshoot
#     return all_peaks #, all_peaks ## don't necessarily need all_peaks array, keep it for troubleshoot


# peakthresh = 2 ## threshold for peaks is estimated
# peak_beg = 1400
# peak_end = 7600
# peak_range_beg = [0, peak_beg] ## hardcoded for 0-1400 and 7600-9000
# peak_range_end = [peak_end, 9000]

# peaks_test_beg, sum_ranges_test_beg = find_peaks(ys_basesub[0], peakthresh, peak_range_beg)
# peaks_test_end, sum_ranges_test_end = find_peaks(ys_basesub[0], peakthresh, peak_range_end)


In [17]:
print(len(ys_basesub))

13


In [18]:
print(len(range(peak_range_beg[0],peak_range_beg[1])))
print(len(range(peak_range_end[0],peak_range_end[1])))

1399
1400


In [19]:
# now try numpy version

start = time.time()
peakthresh = 2 ## threshold for peaks is estimated
peak_beg = 1399
peak_end = 7599
peak_range_beg = [0, peak_beg] ## hardcoded for 0-1400 and 7600-9000
peak_range_end = [peak_end, 8999]

sum_ranges_beg_np = np.zeros((len(ys_basesub[:-1]), len(ys_basesub[0])), dtype=object) ## 13 channels, 13 sequences, added pulses for ON
sum_ranges_end_np = np.zeros((len(ys_basesub[:-1]), len(ys_basesub[0])), dtype=object) ## 13 channels, 13 sequences, added pulses for ON

# sum_ranges_beg = []
# sum_ranges_end = []

for i in range(len(sum_ranges_beg_np)):
#     sum_ranges_beg, peaks_beg = find_peaks(ys_basesub[i], peakthresh, peak_range_beg)
#     sum_ranges_end, peaks_end = find_peaks(ys_basesub[i], peakthresh, peak_range_end)
    print('channel ' + str(chan_enab[i]))
    sum_ranges_beg_np[i] = find_peaks_np(ys_basesub[i], peakthresh, peak_range_beg)
    sum_ranges_end_np[i] = find_peaks_np(ys_basesub[i], peakthresh, peak_range_end)
#     peaks_beg  = find_peaks(ys_basesub[i], peakthresh, peak_range_beg)
#     peaks_end  = find_peaks(ys_basesub[i], peakthresh, peak_range_end)

end = time.time()
print(end-start)

channel 0
channel 1
channel 2
channel 3
channel 4
channel 5
channel 6
channel 7
channel 8
channel 9
channel 10
channel 11
28.401197910308838


In [20]:
print(len(sum_ranges_end_np))
print(len(sum_ranges_end_np[0]))
print(len(sum_ranges_end_np[0][0]))

12
5000
86


In [21]:
# looks like they are the same

print(len(sum_ranges_beg))
print(len(sum_ranges_beg_np))
print(len(sum_ranges_beg[0]))
print(len(sum_ranges_beg_np[0]))
# print((sum_ranges_beg[0][250]))
# print((sum_ranges_beg_np[0][250]))

for i in range(5000):
#     print(i)
    if np.all(sum_ranges_end[0][i] == sum_ranges_end_np[0][i]) == 0:
        print(i)
        print(np.all(sum_ranges_end[0][i] == sum_ranges_end_np[0][i]))
        print(' something wrong hihihihihihihihihihihihi')
        print()
#     print(np.all(sum_ranges_beg[0][i] == sum_ranges_beg_np[0][i]))

#     print()

13
12
5000
5000


In [22]:
# print((sum_ranges_beg[0][25]))
# print((sum_ranges_beg_np[0][25]))
print((sum_ranges_beg[0][626]))
print((sum_ranges_beg_np[0][626]))

[[10, 14], [35, 39], [118, 122], [189, 193], [197, 201], [227, 231], [256, 260], [307, 311], [335, 339], [374, 378], [422, 426], [455, 459], [499, 503], [506, 510], [519, 523], [541, 545], [573, 577], [580, 584], [587, 591], [620, 624], [627, 631], [638, 642], [649, 653], [673, 677], [675, 679], [682, 686], [708, 712], [718, 722], [727, 731], [754, 758], [846, 850], [849, 853], [874, 878], [886, 890], [935, 939], [964, 968], [966, 970], [981, 985], [1011, 1015], [1031, 1035], [1039, 1043], [1045, 1049], [1127, 1131], [1134, 1138], [1149, 1153], [1155, 1159], [1164, 1168], [1184, 1188], [1214, 1218], [1224, 1228], [1248, 1252], [1253, 1257], [1275, 1279], [1284, 1288], [1321, 1325], [1330, 1334], [1332, 1336], [1371, 1375]]
[[  10   14]
 [  35   39]
 [ 118  122]
 [ 189  193]
 [ 197  201]
 [ 227  231]
 [ 256  260]
 [ 307  311]
 [ 335  339]
 [ 374  378]
 [ 422  426]
 [ 455  459]
 [ 499  503]
 [ 506  510]
 [ 519  523]
 [ 541  545]
 [ 573  577]
 [ 580  584]
 [ 587  591]
 [ 620  624]
 [ 627 

In [23]:
print(len(((sum_ranges_beg_np))))
print(len(((sum_ranges_beg_np[0]))))
print(len(((sum_ranges_beg_np[0][4999]))))
# print((((sum_ranges_beg_np))))
# print((((sum_ranges_beg_np[0]))))
print((((sum_ranges_beg_np[0][1][0][0]))))

12
5000
57
3


In [24]:
print(len(ys_basesub))
print(len(ys_basesub[0]))
print(len(ys_basesub[0][0]))
print((ys_basesub[0][0][10]))

13
5000
9000
-0.9188953541470739


try modifying add_pulse function to instead add points in same pulse

In [32]:
def integrate_peaks(ys, int_ranges): ## ranges in which to integrate int_ranges = [[2,6], [6,10]], 2 len array for each point
    peak_integrals = []
    for p in range(len(int_ranges)):  ## find integral of peaks for every pulse p in one detector channel
#         print(p)
        peak_ints = np.zeros((len(int_ranges[p])), dtype = np.float64)
        print(p)
#         print(len(peak_ints))
        for pk in range(len(int_ranges[p])):  ## size of the peaks for every pulse
            start_point = int_ranges[p][pk][0] ## try this to condense code. Basically, the beginning of where to integrate
            end_point = int_ranges[p][pk][1]
#             print(end_point)
            for point in range(start_point,end_point+1): ##From 20-60 for example. SFarr[2] is the array of start to end pulses to sum
                peak_ints[pk] = np.add(peak_ints[pk],ys[p][point]) ## start with zeros, add to each iteratively
        peak_integrals.append(peak_ints)
    return peak_integrals
    

In [33]:
print(ys_basesub[0][1][8999])

5.569524592503707


In [34]:
start = time.time()

ints_beg = np.zeros((len(sum_ranges_beg_np), len(sum_ranges_beg_np[0])), dtype=object) ## 13 channels, 13 sequences, added pulses for ON
ints_end = np.zeros((len(sum_ranges_end_np), len(sum_ranges_end_np[0])), dtype=object) ## 13 channels, 13 sequences, added pulses for ON

for i in range(len(sum_ranges_beg_np)):
    print('channel ' + str(chan_enab[i]))
    ints_beg[i] = integrate_peaks(ys_basesub[i], sum_ranges_beg_np[i])
    ints_end[i] = integrate_peaks(ys_basesub[i], sum_ranges_end_np[i])
    
end = time.time()
print(end-start)

channel 0
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274


2164
2165
2166
2167
2168
2169
2170
2171
2172
2173
2174
2175
2176
2177
2178
2179
2180
2181
2182
2183
2184
2185
2186
2187
2188
2189
2190
2191
2192
2193
2194
2195
2196
2197
2198
2199
2200
2201
2202
2203
2204
2205
2206
2207
2208
2209
2210
2211
2212
2213
2214
2215
2216
2217
2218
2219
2220
2221
2222
2223
2224
2225
2226
2227
2228
2229
2230
2231
2232
2233
2234
2235
2236
2237
2238
2239
2240
2241
2242
2243
2244
2245
2246
2247
2248
2249
2250
2251
2252
2253
2254
2255
2256
2257
2258
2259
2260
2261
2262
2263
2264
2265
2266
2267
2268
2269
2270
2271
2272
2273
2274
2275
2276
2277
2278
2279
2280
2281
2282
2283
2284
2285
2286
2287
2288
2289
2290
2291
2292
2293
2294
2295
2296
2297
2298
2299
2300
2301
2302
2303
2304
2305
2306
2307
2308
2309
2310
2311
2312
2313
2314
2315
2316
2317
2318
2319
2320
2321
2322
2323
2324
2325
2326
2327
2328
2329
2330
2331
2332
2333
2334
2335
2336
2337
2338
2339
2340
2341
2342
2343
2344
2345
2346
2347
2348
2349
2350
2351
2352
2353
2354
2355
2356
2357
2358
2359
2360
2361
2362
2363


4306
4307
4308
4309
4310
4311
4312
4313
4314
4315
4316
4317
4318
4319
4320
4321
4322
4323
4324
4325
4326
4327
4328
4329
4330
4331
4332
4333
4334
4335
4336
4337
4338
4339
4340
4341
4342
4343
4344
4345
4346
4347
4348
4349
4350
4351
4352
4353
4354
4355
4356
4357
4358
4359
4360
4361
4362
4363
4364
4365
4366
4367
4368
4369
4370
4371
4372
4373
4374
4375
4376
4377
4378
4379
4380
4381
4382
4383
4384
4385
4386
4387
4388
4389
4390
4391
4392
4393
4394
4395
4396
4397
4398
4399
4400
4401
4402
4403
4404
4405
4406
4407
4408
4409
4410
4411
4412
4413
4414
4415
4416
4417
4418
4419
4420
4421
4422
4423
4424
4425
4426
4427
4428
4429
4430
4431
4432
4433
4434
4435
4436
4437
4438
4439
4440
4441
4442
4443
4444
4445
4446
4447
4448
4449
4450
4451
4452
4453
4454
4455
4456
4457
4458
4459
4460
4461
4462
4463
4464
4465
4466
4467
4468
4469
4470
4471
4472
4473
4474
4475
4476
4477
4478
4479
4480
4481
4482
4483
4484
4485
4486
4487
4488
4489
4490
4491
4492
4493
4494
4495
4496
4497
4498
4499
4500
4501
4502
4503
4504
4505


1214
1215
1216
1217
1218
1219
1220
1221
1222
1223
1224
1225
1226
1227
1228
1229
1230
1231
1232
1233
1234
1235
1236
1237
1238
1239
1240
1241
1242
1243
1244
1245
1246
1247
1248
1249
1250
1251
1252
1253
1254
1255
1256
1257
1258
1259
1260
1261
1262
1263
1264
1265
1266
1267
1268
1269
1270
1271
1272
1273
1274
1275
1276
1277
1278
1279
1280
1281
1282
1283
1284
1285
1286
1287
1288
1289
1290
1291
1292
1293
1294
1295
1296
1297
1298
1299
1300
1301
1302
1303
1304
1305
1306
1307
1308
1309
1310
1311
1312
1313
1314
1315
1316
1317
1318
1319
1320
1321
1322
1323
1324
1325
1326
1327
1328
1329
1330
1331
1332
1333
1334
1335
1336
1337
1338
1339
1340
1341
1342
1343
1344
1345
1346
1347
1348
1349
1350
1351
1352
1353
1354
1355
1356
1357
1358
1359
1360
1361
1362
1363
1364
1365
1366
1367
1368
1369
1370
1371
1372
1373
1374
1375
1376
1377
1378
1379
1380
1381
1382
1383
1384
1385
1386
1387
1388
1389
1390
1391
1392
1393
1394
1395
1396
1397
1398
1399
1400
1401
1402
1403
1404
1405
1406
1407
1408
1409
1410
1411
1412
1413


3137
3138
3139
3140
3141
3142
3143
3144
3145
3146
3147
3148
3149
3150
3151
3152
3153
3154
3155
3156
3157
3158
3159
3160
3161
3162
3163
3164
3165
3166
3167
3168
3169
3170
3171
3172
3173
3174
3175
3176
3177
3178
3179
3180
3181
3182
3183
3184
3185
3186
3187
3188
3189
3190
3191
3192
3193
3194
3195
3196
3197
3198
3199
3200
3201
3202
3203
3204
3205
3206
3207
3208
3209
3210
3211
3212
3213
3214
3215
3216
3217
3218
3219
3220
3221
3222
3223
3224
3225
3226
3227
3228
3229
3230
3231
3232
3233
3234
3235
3236
3237
3238
3239
3240
3241
3242
3243
3244
3245
3246
3247
3248
3249
3250
3251
3252
3253
3254
3255
3256
3257
3258
3259
3260
3261
3262
3263
3264
3265
3266
3267
3268
3269
3270
3271
3272
3273
3274
3275
3276
3277
3278
3279
3280
3281
3282
3283
3284
3285
3286
3287
3288
3289
3290
3291
3292
3293
3294
3295
3296
3297
3298
3299
3300
3301
3302
3303
3304
3305
3306
3307
3308
3309
3310
3311
3312
3313
3314
3315
3316
3317
3318
3319
3320
3321
3322
3323
3324
3325
3326
3327
3328
3329
3330
3331
3332
3333
3334
3335
3336


...

1561
1562
1563
1564
1565
1566
1567
1568
1569
1570
1571
1572
1573
1574
1575
1576
1577
1578
1579
1580
1581
1582
1583
1584
1585
1586
1587
1588
1589
1590
1591
1592
1593
1594
1595
1596
1597
1598
1599
1600
1601
1602
1603
1604
1605
1606
1607
1608
1609
1610
1611
1612
1613
1614
1615
1616
1617
1618
1619
1620
1621
1622
1623
1624
1625
1626
1627
1628
1629
1630
1631
1632
1633
1634
1635
1636
1637
1638
1639
1640
1641
1642
1643
1644
1645
1646
1647
1648
1649
1650
1651
1652
1653
1654
1655
1656
1657
1658
1659
1660
1661
1662
1663
1664
1665
1666
1667
1668
1669
1670
1671
1672
1673
1674
1675
1676
1677
1678
1679
1680
1681
1682
1683
1684
1685
1686
1687
1688
1689
1690
1691
1692
1693
1694
1695
1696
1697
1698
1699
1700
1701
1702
1703
1704
1705
1706
1707
1708
1709
1710
1711
1712
1713
1714
1715
1716
1717
1718
1719
1720
1721
1722
1723
1724
1725
1726
1727
1728
1729
1730
1731
1732
1733
1734
1735
1736
1737
1738
1739
1740
1741
1742
1743
1744
1745
1746
1747
1748
1749
1750
1751
1752
1753
1754
1755
1756
1757
1758
1759
1760


3238
3239
3240
3241
3242
3243
3244
3245
3246
3247
3248
3249
3250
3251
3252
3253
3254
3255
3256
3257
3258
3259
3260
3261
3262
3263
3264
3265
3266
3267
3268
3269
3270
3271
3272
3273
3274
3275
3276
3277
3278
3279
3280
3281
3282
3283
3284
3285
3286
3287
3288
3289
3290
3291
3292
3293
3294
3295
3296
3297
3298
3299
3300
3301
3302
3303
3304
3305
3306
3307
3308
3309
3310
3311
3312
3313
3314
3315
3316
3317
3318
3319
3320
3321
3322
3323
3324
3325
3326
3327
3328
3329
3330
3331
3332
3333
3334
3335
3336
3337
3338
3339
3340
3341
3342
3343
3344
3345
3346
3347
3348
3349
3350
3351
3352
3353
3354
3355
3356
3357
3358
3359
3360
3361
3362
3363
3364
3365
3366
3367
3368
3369
3370
3371
3372
3373
3374
3375
3376
3377
3378
3379
3380
3381
3382
3383
3384
3385
3386
3387
3388
3389
3390
3391
3392
3393
3394
3395
3396
3397
3398
3399
3400
3401
3402
3403
3404
3405
3406
3407
3408
3409
3410
3411
3412
3413
3414
3415
3416
3417
3418
3419
3420
3421
3422
3423
3424
3425
3426
3427
3428
3429
3430
3431
3432
3433
3434
3435
3436
3437


199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
408
409
410
411
412
413
414
415
416
417
418
419
420
421
422
423
424
425
426
427
428
429
430
431
432
433
434
435
436
437
438
439
440
441
442
443
444
445
446
447
448


2591
2592
2593
2594
2595
2596
2597
2598
2599
2600
2601
2602
2603
2604
2605
2606
2607
2608
2609
2610
2611
2612
2613
2614
2615
2616
2617
2618
2619
2620
2621
2622
2623
2624
2625
2626
2627
2628
2629
2630
2631
2632
2633
2634
2635
2636
2637
2638
2639
2640
2641
2642
2643
2644
2645
2646
2647
2648
2649
2650
2651
2652
2653
2654
2655
2656
2657
2658
2659
2660
2661
2662
2663
2664
2665
2666
2667
2668
2669
2670
2671
2672
2673
2674
2675
2676
2677
2678
2679
2680
2681
2682
2683
2684
2685
2686
2687
2688
2689
2690
2691
2692
2693
2694
2695
2696
2697
2698
2699
2700
2701
2702
2703
2704
2705
2706
2707
2708
2709
2710
2711
2712
2713
2714
2715
2716
2717
2718
2719
2720
2721
2722
2723
2724
2725
2726
2727
2728
2729
2730
2731
2732
2733
2734
2735
2736
2737
2738
2739
2740
2741
2742
2743
2744
2745
2746
2747
2748
2749
2750
2751
2752
2753
2754
2755
2756
2757
2758
2759
2760
2761
2762
2763
2764
2765
2766
2767
2768
2769
2770
2771
2772
2773
2774
2775
2776
2777
2778
2779
2780
2781
2782
2783
2784
2785
2786
2787
2788
2789
2790


4667
4668
4669
4670
4671
4672
4673
4674
4675
4676
4677
4678
4679
4680
4681
4682
4683
4684
4685
4686
4687
4688
4689
4690
4691
4692
4693
4694
4695
4696
4697
4698
4699
4700
4701
4702
4703
4704
4705
4706
4707
4708
4709
4710
4711
4712
4713
4714
4715
4716
4717
4718
4719
4720
4721
4722
4723
4724
4725
4726
4727
4728
4729
4730
4731
4732
4733
4734
4735
4736
4737
4738
4739
4740
4741
4742
4743
4744
4745
4746
4747
4748
4749
4750
4751
4752
4753
4754
4755
4756
4757
4758
4759
4760
4761
4762
4763
4764
4765
4766
4767
4768
4769
4770
4771
4772
4773
4774
4775
4776
4777
4778
4779
4780
4781
4782
4783
4784
4785
4786
4787
4788
4789
4790
4791
4792
4793
4794
4795
4796
4797
4798
4799
4800
4801
4802
4803
4804
4805
4806
4807
4808
4809
4810
4811
4812
4813
4814
4815
4816
4817
4818
4819
4820
4821
4822
4823
4824
4825
4826
4827
4828
4829
4830
4831
4832
4833
4834
4835
4836
4837
4838
4839
4840
4841
4842
4843
4844
4845
4846
4847
4848
4849
4850
4851
4852
4853
4854
4855
4856
4857
4858
4859
4860
4861
4862
4863
4864
4865
4866


In [35]:
print(range(8999-4,8999))
for i in (range(8999-4,8999)):
    print(i)
print(sum_ranges_end_np[0][1][-1])

range(8995, 8999)
8995
8996
8997
8998
[8995 8999]


In [37]:
p = 0
print((sum_ranges_end_np[0][p][0]))
start = (sum_ranges_end_np[0][p][1][0])
end = (sum_ranges_end_np[0][p][1][1])
print(end)

print()

sums = [ys_basesub[0][p][start], ys_basesub[0][p][start+1], ys_basesub[0][p][start+2], ys_basesub[0][p][start+3], ys_basesub[0][p][start+4]]
print(ys_basesub[0][p][start])
print(ys_basesub[0][p][start+1])
print(ys_basesub[0][p][start+2])
print(ys_basesub[0][p][start+3])
print(ys_basesub[0][p][start+4])
print()
for i in range(start,end+1):
    print(ys_basesub[0][p][i])

# print(len(ints_end[0][p]))
print(len(sum_ranges_end_np[0][p]))

print(sum(sums))
print(ints_end[0][p])

[7631 7635]
7641

-0.18720755082213802
0.3981426918378107
7.568683164422183
0.5444802525027979
0.2518051311728235

-0.18720755082213802
0.3981426918378107
7.568683164422183
0.5444802525027979
0.2518051311728235
86
8.575903689113476
[ 2.5760637   8.57590369 24.96571048 58.91602456  5.79549004 14.72208124
 12.23434271 21.16093391 30.08752511  5.06380223  5.06380223 11.5026549
  6.08816516  9.89294174 46.62366946  8.86857881 20.57558366  9.30759149
  8.28322857 12.81969295  9.01491637  3.74676419 25.2583856  43.98959337
 45.7456441  31.69723827 20.72192122 21.89262171  5.79549004 13.40504319
 12.38068027 13.55138075  4.33211443  5.35647735 24.08768512 11.79533002
  5.64915248  5.21013979 52.47717189 17.50249489 62.57446357 15.6001066
 31.84357583 13.55138075  4.77112711  6.96619052 22.91698463 17.94150757
 12.38068027  8.28322857  7.69787833 52.76984701 12.81969295  6.52717784
  4.47845199  6.52717784  5.06380223 11.64899246 37.5507407  19.6975583
  9.45392905 15.74644416 10.18561686 14.1

### Below plots peaks and their widths

In [23]:
# print(np.where(peaks<1800))
x = ys_basesub[0][7][2:-2].copy()
# x = ys_basesub[0][100].copy()

print(x)
print(len(x))

print(np.mean(x[0:1990]))
peakthresh = 2 ## threshold for peaks is estimated
peak_beg = 1400
peak_end = 7600

peaks, _ = sp.signal.find_peaks(x, threshold=peakthresh)#, height  = [1,1000])

peaks_beg_i = np.array(np.where(peaks<peak_beg)[0], copy=True)
peaks_end_i = np.array(np.where(peaks>peak_end)[0], copy=True)
peaks_beg_ys = x[peaks][peaks_beg_i] ## think this is wrong?
peaks_end_ys = x[peaks][peaks_end_i]

# print(len(peaks_beg_i))
# print(len(peaks_end_i))
# peaks[0] = 0
print(peaks)
print(peaks_beg_i)
print(x[peaks[peaks_beg_i]])
print(x[peaks][0:46])


[ 0.28042947 -0.30972649 -0.60480448 ... -0.01464851  0.13289048
  0.13289048]
8996
-0.0237677581536971
[   5   45   58   60   66   97  110  121  159  166  243  276  300  343
  347  397  400  405  431  462  465  478  486  532  561  566  571  675
  734  780  795  834  875  883  898  904  982  995 1026 1034 1039 1067
 1073 1100 1155 1168 1172 1176 1205 1213 1227 1268 1300 1307 1378 1401
 1437 1523 1526 1563 1585 1620 1638 1677 1685 1706 1718 1746 1751 1786
 1793 1828 1832 1876 1963 1981 2043 2047 2049 2052 2055 2057 2059 2061
 2065 2068 2074 2076 2084 2086 2088 2092 2096 2098 2104 2106 2109 2114
 2116 2118 2121 2126 2131 2137 2139 2141 2143 2146 2150 2153 2156 2158
 2161 2163 2166 2169 2173 2175 2177 2180 2185 2187 2192 2194 2197 2200
 2202 2204 2207 2209 2212 2216 2220 2224 2226 2230 2232 2234 2240 2242
 2245 2249 2252 2255 2261 2264 2269 2275 2277 2280 2283 2290 2293 2296
 2300 2307 2309 2315 2318 2321 2325 2328 2333 2336 2339 2341 2345 2348
 2354 2356 2359 2363 2366 2368 2373 2375 238

In [89]:
peaks_w_beg = sp.signal.peak_widths(x, peaks[peaks_beg_i], rel_height=0.8)
peaks_w_end = sp.signal.peak_widths(x, peaks[peaks_end_i], rel_height=0.8)

# results_full = sp.signal.peak_widths(x, peaks, rel_height=1)

plt.plot(x, label='Example NaI pulse')

# plt.plot(peaks[peaks_beg_i], x[peaks[peaks_beg_i]], "x")
plt.plot(peaks[peaks_end_i], x[peaks[peaks_end_i]], "x", label = 'Single gamma region peak')

# plt.hlines(*peaks_w_beg[1:], color="C2")
plt.hlines(*peaks_w_end[1:], color="C3", label = 'Peak\'s width')


plt.title('Gamma peak finding') 
# plt.legend(['NaI pulse', 'peak', 'begin integrating here?', 'max height threshold'])
plt.legend()
plt.xlabel('bin')
plt.ylabel('ADC')
plt.show()
# plt.hlines(*results_full[1:], color="C3")

plt.show()

In [141]:

x = ys_basesub[0][2].copy()
peakthresh = 2 ## threshold for peaks is estimated
peak_beg = 1401
peak_end = 7600

peaks, _ = sp.signal.find_peaks(x, threshold=peakthresh)#, height  = [1,1000])

peaks_beg_i = np.array(np.where(peaks<peak_beg)[0], copy=True)
peaks_end_i = np.array(np.where(peaks>peak_end)[0], copy=True)
peaks_beg_ys = x[peaks][peaks_beg_i]
peaks_end_ys = x[peaks][peaks_end_i]

# peaks_beg_i = np.array(np.where(peaks<1700)[0], copy=True)
peaks_end_i = np.array(np.where(peaks>peak_end)[0], copy=True)

# peaks_beg_ys = x[peaks][peaks_beg_i]
peaks_end_ys = x[peaks][peaks_beg_i]

plt.plot(x, label='NaI pulse')

# plt.plot(peaks, x[peaks], "x", label='peaks')
plt.plot(peaks[peaks_beg_i], x[peaks[peaks_beg_i]], ".", label='Pre-trigger region peaks')
plt.plot(peaks[peaks_end_i], x[peaks[peaks_end_i]], "mx", label='"pulse mode" region gamma peaks')
plt.axvline(peak_end, ls = '--', c ='r', label='Begin "pulse mode" region')
# plt.axhline(mingheight, ls = '--', c ='g', label='above noise floor threshold')
# plt.axhline(maxgheight, ls = '--', c ='g', label='max height threshold?')

# plt.xlim(7000,9000)
# plt.ylim(-20,500)

plt.title('gamma peak finding') 
# plt.legend(['NaI pulse', 'peak', 'begin integrating here?', 'max height threshold'])
plt.legend()
plt.xlabel('bin')
plt.ylabel('ADC')
plt.show()
# print(peaks)
print(peaks[peaks_beg_i])
print(peaks[peaks_end_i])

[  22  100  109  127  213  225  228  235  281  317  331  341  352  372
  380  389  414  456  501  509  546  554  562  577  582  596  608  648
  660  665  707  791  813  816  863  889  973  988 1010 1034 1055 1159
 1178 1195 1208 1220 1244 1258 1268 1281 1283 1291 1294 1312 1314 1348
 1350]
[7602 7610 7619 7638 7672 7695 7698 7709 7730 7767 7781 7807 7824 7841
 7871 7890 7900 7908 7916 7936 7961 7975 7983 7999 8009 8016 8044 8060
 8102 8112 8117 8134 8158 8194 8210 8217 8241 8247 8254 8261 8272 8285
 8291 8302 8325 8371 8409 8412 8416 8422 8428 8433 8445 8457 8500 8504
 8506 8514 8524 8533 8550 8553 8558 8562 8578 8591 8600 8602 8614 8633
 8655 8660 8676 8681 8685 8693 8731 8738 8747 8767 8837 8841 8877 8882
 8889 8906 8909 8911 8945 8958 8962 8975 8998]


In [129]:
# print(results_half[0])
print(np.mean(peaks_w_beg[0]))
print(np.mean(peaks_w_end[0]))

2.1645910417818954


In [43]:
## I think this has been confirmed to work using the singular finding method done earlier.

def find_peaks(ys, pthresh, peakrange): ## ys here is for one channel!
#     pulse_arr = [[],[]] ## peaks index, pulse sum ranges
    all_peaks = []
    sum_ranges = []  ## all sum ranges  (i.e. for peak at bin 7, sum from bins 5-9. Save this for every peak)
    for p in range(len(ys)):  ## find peaks for every pulse p in one detector channel
        p_sum_ranges = []  ## one array of all sum ranges for a pulse p
        peaks, _ = sp.signal.find_peaks(ys[p], threshold=pthresh)#, height  = [1,1000])
        temp_i = np.array(np.where((peaks>=peakrange[0]) & (peaks<=peakrange[1]))[0], copy=True)
        peaks_i = peaks[temp_i]
        all_peaks.append(peaks_i)
        for peak in peaks_i:
            if peak-2<peakrange[0]: ## plus and minus 2 x points (512 ns each) was determined from peak widths below
#                 print('peak in lower bound at pulse ' +str(p) +', peak '+str(peak))
                pass
            elif peak+2>peakrange[1]:
#                 print('pulse ' +str(p))
#                 print('peak in upper bound at pulse ' +str(p) +', peak '+str(peak))
                pass
            else:
                pm_range = [peak-2, peak+2]  #plus minus 2 bin range around found peak
                p_sum_ranges.append(pm_range)
        sum_ranges.append(p_sum_ranges)
    return all_peaks, sum_ranges ## don't necessarily need all_peaks array, keep it for troubleshoot

peakthresh = 2 ## threshold for peaks is estimated
peak_beg = 1400
peak_end = 7600
peak_range_beg = [0, peak_beg] ## hardcoded for 0-1400 and 7600-9000
peak_range_end = [peak_end, 9000]

peaks_test_beg, sum_ranges_test_beg = find_peaks(ys_basesub[0], peakthresh, peak_range_beg)
peaks_test_end, sum_ranges_test_end = find_peaks(ys_basesub[0], peakthresh, peak_range_end)


In [40]:
for i in range(len(ys_basesub)):
    peaks_test_beg, sum_ranges_test_beg = find_peaks(ys_basesub[0], peakthresh, peak_range_beg)
    peaks_test_end, sum_ranges_test_end = find_peaks(ys_basesub[0], peakthresh, peak_range_end)

KeyboardInterrupt: 

In [51]:
print((peaks_test_end[250]))
print(all(peaks_test_beg[250] == (peaks[peaks_beg_i])))
print(all(peaks_test_end[250] == (peaks[peaks_end_i])))
# for i in peaks_test_end[16]:
#     print( i = (peaks[peaks_end_i]))
# print(sum_ranges_test_beg)
print(sum_ranges_test_end[250])
# len(peaks_test_beg[3])

[7611 7616 7633 7687 7703 7706 7725 7747 7773 7790 7804 7815 7835 7859
 7874 7890 7901 7907 7943 7945 7951 7960 7977 8025 8027 8049 8058 8077
 8087 8108 8125 8163 8183 8241 8250 8303 8314 8352 8358 8372 8389 8392
 8420 8465 8489 8532 8550 8552 8627 8653 8671 8680 8682 8700 8729 8740
 8742 8755 8761 8784 8787 8789 8802 8811 8841 8856 8865 8886 8911 8925
 8929 8942 8954 8967 8980]
True
True
[[7609, 7613], [7614, 7618], [7631, 7635], [7685, 7689], [7701, 7705], [7704, 7708], [7723, 7727], [7745, 7749], [7771, 7775], [7788, 7792], [7802, 7806], [7813, 7817], [7833, 7837], [7857, 7861], [7872, 7876], [7888, 7892], [7899, 7903], [7905, 7909], [7941, 7945], [7943, 7947], [7949, 7953], [7958, 7962], [7975, 7979], [8023, 8027], [8025, 8029], [8047, 8051], [8056, 8060], [8075, 8079], [8085, 8089], [8106, 8110], [8123, 8127], [8161, 8165], [8181, 8185], [8239, 8243], [8248, 8252], [8301, 8305], [8312, 8316], [8350, 8354], [8356, 8360], [8370, 8374], [8387, 8391], [8390, 8394], [8418, 8422], [8463

61

In [ ]:
print

In [28]:
# print(x[2000:-1])
x = ys_basesub[0][100].copy()

initpeaks, _ = sp.signal.find_peaks(x, height=[10,250], threshold=10, width=1)
init_peaks_beg_i = np.array(np.where(initpeaks<1700)[0], copy=True)
init_peaks_beg_ys = x[initpeaks][init_peaks_beg_i]

mingstd = np.std(init_peaks_beg_ys)
mingavg = np.mean(init_peaks_beg_ys)
gmax = (mingavg+(2*mingstd))
print(mingstd)
print(gmax)
plt.hist(x[initpeaks][init_peaks_beg_i])
plt.axvline(gmax, ls = '--', c ='r')

mingheight = np.mean(init_peaks_beg_ys[np.where(init_peaks_beg_ys<gmax)])

print(mingavg)
print(mingheight)

plt.title('"Noise floor" of beginning region') 
plt.legend(['2 std within the mean = ' + str(round(gmax, 2)) + '\n this represents the cutoff region for noisefloor average', 'peaks distribution'])
plt.xlabel('bin')
plt.ylabel('ADC')
plt.show()

1.6370690310889247
18.043690131798165
14.769552069620318
14.769552069620318


use 6Li t0 for all instead of for themselves individually ##

In [ ]:
start = time.time()

In [20]:
!jupyter --version

Selected Jupyter core packages...
IPython          : 7.31.1
ipykernel        : 6.15.2
ipywidgets       : 7.6.5
jupyter_client   : 7.3.4
jupyter_core     : 4.11.1
jupyter_server   : 1.18.1
jupyterlab       : 3.4.4
nbclient         : 0.5.13
nbconvert        : 6.4.4
nbformat         : 5.5.0
notebook         : 6.4.12
qtconsole        : 5.2.2
traitlets        : 5.1.1


In [ ]:
NaIthresh=2000
Li6thresh=1000

In [ ]:
threshold_array = (np.full(len(ys_basesub), NaIthresh))
threshold_array[-1] = Li6thresh

njit ## numba does not support reversed, but this could be changed if it's slow

In [ ]:
def find_offset(ys, thresharr):
    xCrosses = np.zeros((len(ys), numRuns)) #outer array is crossing arrays for given channel, inner array is crossing for each event
    offset = np.zeros((len(ys), numRuns), dtype=np.int32) ##offset in bins for each channel, each pulse
    modeCrosses = np.zeros((len(ys)), dtype=np.float64)
    for i in reversed(range(len(ys))):
        #xValues.append([])
        for p in range(len(ys[i])):
            xing = np.argmax(ys[i][p] > thresharr[i])
            #print(xing)
            xCrosses[i][p] = xing
        modeCrosses[i] = (st.mode(xCrosses[i])) #find the most typical crossing value for each channel
        for p in range(len(xCrosses[i])):
            offset[i][p] = (modeCrosses[-1] - xCrosses[i][p]) ## make sure this is the correct sign!!! 
    if (np.all(xCrosses[-1])) == False:
        emessage = ('ERROR: 6Li threshold was not reached for at least one pulse')
        logger.error(run_num + emessage)
        raise Exception(emessage)
    return offset, xCrosses, modeCrosses
                           
offset, xCrosses, modeCrosses = find_offset(ys_basesub, threshold_array)

In [ ]:
end = time.time()
# print('finding offset time: ' + str(end-start))  

In[11]:

 this cell loops through every channel as opposed to inputting all channels at once. 5x faster ##

In [ ]:
start = time.time()

extend all arrays by a value, check that the max number of offset on 6Li is less than that value ##

In [ ]:
extendedRange = 3 ## must be a positive value which to extend ys_arr
if abs(max(offset[-1], key = abs)) > extendedRange: ## if the max offset of 6Li is >extendedRange, something is wrong
    emessage = ('ERROR: largest offset greater than extended range')
    logger.error(run_num + emessage)
    raise Exception(emessage)

Below commented out for testing without t0 aligning 08.08.24 ##

In [ ]:
try:
    ys_ext = np.zeros((len(ys_basesub), len(ys_basesub[0]), len(ys_basesub[0][0])+extendedRange*2), dtype=np.float64)
    ys_cut = np.zeros((len(ys_basesub), len(ys_basesub[0]), (len(ys_ext[0][0])-((extendedRange*2)+1)*2)))
    xs_cut = np.zeros((len(ys_cut), len(ys_cut[0][0])))
except Exception as e:
    logger.error(run_num + ' failed during ys_cut array creation')
    logger.exception(e)

cant use jit because np.pad is not supported ##

In [ ]:
def align_cut(ys, xs_arr, extendedr):
    tempys_ext = np.zeros((len(ys), len(ys[0])+extendedr*2), dtype=np.float64)
    tempys_cut = np.zeros((len(ys), (len(tempys_ext[0])-((extendedr*2)+1)*2)))
    tempxs_cut = np.zeros(len(tempys_cut[0]))
    for p in range(len(ys)):
        tempys_ext[p] = np.pad(ys[p], extendedr, 'constant', constant_values=(0))
        tempys_ext[p] = np.roll(tempys_ext[p],offset[-1][p]) ## assumes 6Li at -1 position
        tempys_cut[p] = tempys_ext[p][((extendedr*2)+1):-((extendedr*2)+1)].copy() ## cut by 7 (if extRange == 3)
        tempys_cut[p] = tempys_cut[p]/HeNorms[p] ## normalize by 3He integral  ## comment out if using basesub_norm
    x_cut_amt = int((len(ys[0]) - len(tempys_cut[0]))/2)
    tempxs_cut = xs_arr[x_cut_amt:-x_cut_amt].copy()
    return tempys_cut, tempxs_cut

Below commented out for testing without t0 aligning 08.08.24 ##

In [ ]:
try:
    for i in range(len(ys_basesub)):
        ys_cut[i], xs_cut[i] = align_cut(ys_basesub[i], xs[i], extendedRange)
except Exception as e:
    logger.error(run_num + ' failed aligning and cutting')
    logger.exception(e)
    
# checkp = 2053
# print(offset[-1][checkp]) ## checking offset for one example checkpulse
# print('original index for checkpulse: '+str(np.argmax(ys_basesub[0][checkp]> 2000))) ## we can follow the index as it changes with extension/cut
# #print('extended range index for checkpulse: '+str(np.argmax(ys_ext[0][checkp]> 2000)))
# print('cut array index for checkpulse: '+str(np.argmax((ys_cut[0][checkp]*HeNorms[checkp])> 2000)))

In [ ]:
del ys_ext ## might help with memory issues
del ys_basesub

In [ ]:
end = time.time()
print('aligning and cutting time: ' + str(end-start))            

In[7]:

In [ ]:
def organize_SF(SFsort_info): ## sometimes pulse 0 has the state switch. In that case, need to account by if clauses below
    counter = 0
    seq = 0
    seq_arr = ([[],[],[]])
    smallerseq = []
    smallerstateis = []
    for i in range(len(SFsort_info[1])-(np.mod((len(SFsort_info[1])), 8))):  ##111 mod 8 = 7, so essentially 111-7 = 104
        counter = counter+1
        if counter < 8:
            if (SF_Sort_arr[1][i]) == 0: ## catches state switches at pulse 0
                smallerstateis.append([(SFsort_info[1][i])+5,(SFsort_info[1][i+1])])
                smallerseq.append(SFsort_info[0][i+1])
                seq = seq+1
                continue
            smallerstateis.append([(SFsort_info[1][i])+5,(SFsort_info[1][i+1])])
            smallerseq.append(SFsort_info[0][i+1])
        elif counter == 8:
            if ((SF_Sort_arr[1][i])+45) >= 5000: ## breaks for state switches at pulse 0
                print(((SF_Sort_arr[1][i])+5))
                seq = seq+1
                seq_arr[0].append(seq)
                seq_arr[1].append(smallerseq)   
                seq_arr[2].append(smallerstateis)
                seq_arr[0] = [x-1 for x in seq_arr[0]] ## reset so sequences are 1-14 instead of 2-15
                break
            seq = seq+1 ## otherwise continue regular sorting
            smallerstateis.append([(SFsort_info[1][i])+5,(SFsort_info[1][i+1])])
            smallerseq.append(SFsort_info[0][i+1])
            seq_arr[0].append(seq)
            seq_arr[1].append(smallerseq)   
            seq_arr[2].append(smallerstateis)
            smallerseq = []
            smallerstateis = []
            counter  = 0
    return seq_arr

In [ ]:
def find_leftover(SFsort_info, seq_arr): ## in case we want to use the other 6 states left over
    left = [[seq_arr[0][-1]+1],[],[]]
    counter = 0
    for i in range((len(SFsort_info[1])-(np.mod((len(SFsort_info[1])), 8))), len(SFsort_info[1])-1):
        counter = counter+1
        if counter < 8:
            left[1].append(SFsort_info[0][i+1])
            left[2].append([(SFsort_info[1][i])+5,(SFsort_info[1][i+1])])
    return left

In [ ]:
try:
    sequence = organize_SF(SF_Sort_arr)
    if len(sequence[0]) == 14: ## catches state switches at pulse 0, leftovers are at the end of the regular sequence
        leftovers = [[sequence[0][-1]],[sequence[1][-1]],[sequence[2][-1]]]
        for i in range(len(sequence)):
            sequence[i].pop(-1) ## deletes the leftovers sequence for state switches at pulse 0
    else:
        leftovers = find_leftover(SF_Sort_arr, sequence) ## otherwise can use normal function
except Exception as e:
    logger.error(run_num + ' failed during sequencing')
    logger.exception(e)

print('sequences '+str(sequence[0]))

In [ ]:
print(str(len(sequence[0]))+' sequences with sequence order: '+str(sequence[1][0]))
# print(leftovers)

In[46]:

 add up pulses for their respective state, in each 8 step sequence ## 
 turning into a by-channel function 06.13.24 ##

In [ ]:
start = time.time()
sequence = np.asarray(sequence, dtype = object)

added_pulses = np.zeros((len(ys_cut), len(sequence[0]), 8, len(ys_cut[0][0])), dtype=np.float64) ## 13 sequences, 8 stages each works?  
i channels, 13 sequences each, 8 states each sequence, 8992 num points ##

In [ ]:
ON_sums = np.zeros((len(ys_cut), len(sequence[0]), len(ys_cut[0][0])), dtype=np.float64) ## 13 channels, 13 sequences, added pulses for ON
OFF_sums = np.zeros((len(ys_cut), len(sequence[0]), len(ys_cut[0][0])), dtype=np.float64) ## 13 channels, 13 sequences, added pulses for OFF
ON_minus_sums = np.zeros((len(ys_cut), len(sequence[0]),2, len(ys_cut[0][0])), dtype=np.float64) ## 13 channels, 13 sequences, added pulses for ON
ON_plus_sums = np.zeros((len(ys_cut), len(sequence[0]),2, len(ys_cut[0][0])), dtype=np.float64) ## 13 channels, 13 sequences, added pulses for OFF

In [ ]:
ON_OFF_sums = np.zeros((len(ys_cut), len(sequence[0]), 2, len(ys_cut[0][0])), dtype=np.float64) ## 13 sequences, 2 for ON or OFF for each sequence 
ON_sums = np.zeros((len(ys_basesub), len(sequence[0]), len(ys_basesub[0][0])), dtype=np.float64) ## 13 channels, 13 sequences, added pulses for ON 
OFF_sums = np.zeros((len(ys_basesub), len(sequence[0]), len(ys_basesub[0][0])), dtype=np.float64) ## 13 channels, 13 sequences, added pulses for OFF 
ON_minus_sums = np.zeros((len(ys_basesub), len(sequence[0]),2, len(ys_basesub[0][0])), dtype=np.float64) ## 13 channels, 13 sequences, added pulses for ON 
ON_plus_sums = np.zeros((len(ys_basesub), len(sequence[0]),2, len(ys_basesub[0][0])), dtype=np.float64) ## 13 channels, 13 sequences, added pulses for OFF 
ON_minus/plus_sums have their 2 for associated OFF states in the second array

@njit

In [ ]:
def add_pulse(ys, SFarr):
#     tempadded_p = np.zeros((len(SFarr[0]), 8, len(ys[0])), dtype=np.float64)
    temp_ON = np.zeros((len(SFarr[0]), len(ys[0])), dtype=np.float64)
    temp_OFF = np.zeros((len(SFarr[0]), len(ys[0])), dtype=np.float64)
    temp_ON_min = np.zeros((len(SFarr[0]),2, len(ys[0])), dtype=np.float64)
    temp_ON_plu = np.zeros((len(SFarr[0]),2, len(ys[0])), dtype=np.float64)
    for seq in range(0, len(SFarr[0])): ## for every sequence
    #         print('seq:' +str(SFarr[0][seq]))
#         print('seq:' +str(seq))
        for state in range(0, len(SFarr[1][0])): ## for every state in the sequence
    #         print("states loop " + str(range(0, len(SFarr[1][0]))[0]) + ' - ' +  str(range(0, len(SFarr[1][0]))[-1]))
            s = SFarr[1][seq][state] ## try this to condense code. Basically, the state currently at
            if s==0 or s==3 or s==5 or s==6: ## these are ON states
#                 print('ON "s" state '+str(s))
#                 print('"state" ' +str(state) + ' from ' + str(range((SFarr[2][seq][state][0]),(SFarr[2][seq][state][1]))))
#                 print('sums from '+str(range((SFarr[2][seq][state][0]),(SFarr[2][seq][state][1]))[0]) +
#                 ' - ' +str(range((SFarr[2][seq][state][0]),(SFarr[2][seq][state][1]))[-1]) + '\n')
                for p in range((SFarr[2][seq][state][0]),(SFarr[2][seq][state][1])): ##From 20-60 for example. SFarr[2] is the array of start to end pulses to sum
                    temp_ON[seq] = np.add(temp_ON[seq],ys[p]) ## start with zeros, add to each iteratively
#                     print(p)
                #         below is for splitting up into ON(-) and ON(+) and their associated OFF states (1 state forward for -, 1 state backward for +)
                if s==0 or s==6: ## these or ON(-) states
                    OFF_i = np.where(np.asarray(SFarr[1][0])==s+1)[0][0] ## find where in sequence state = s(ON-)+1    
    #                 print('ON(-) "s" ' +str(s) + ' from ' + str(range((SFarr[2][seq][state][0]),(SFarr[2][seq][state][1]))))
                    for p in range((SFarr[2][seq][state][0]),(SFarr[2][seq][state][1])): ##From 20-60 for example. SFarr[2] is the array of start to end pulses to sum
                        temp_ON_min[seq][0] = np.add(temp_ON_min[seq][0],ys[p]) ## start with zeros, add to each iteratively
    #                 print('OFF state ' + str(SFarr[1][seq][OFF_i]))
                    for p in range((SFarr[2][seq][OFF_i][0]),(SFarr[2][seq][OFF_i][1])): ##From 20-60 for example. SFarr[2] is the array of start to end pulses to sum
                        temp_ON_min[seq][1] = np.add(temp_ON_min[seq][1],ys[p]) ## start with zeros, add to each iteratively
                if s==3 or s==5: ## these or ON(+) states
                    OFF_i = np.where(np.asarray(SFarr[1][0])==s-1)[0][0] ## find where in sequence state = s(ON+)-1    
    #                 print('ON(+) "s" ' +str(s) + ' from ' + str(range((SFarr[2][seq][state][0]),(SFarr[2][seq][state][1]))))
                    for p in range((SFarr[2][seq][state][0]),(SFarr[2][seq][state][1])): ##From 20-60 for example. SFarr[2] is the array of start to end pulses to sum
                        temp_ON_plu[seq][0] = np.add(temp_ON_plu[seq][0],ys[p]) ## start with zeros, add to each iteratively
    #                 print('OFF state ' + str(SFarr[1][seq][OFF_i]))
                    for p in range((SFarr[2][seq][OFF_i][0]),(SFarr[2][seq][OFF_i][1])): ##From 20-60 for example. SFarr[2] is the array of start to end pulses to sum
                        temp_ON_plu[seq][1] = np.add(temp_ON_plu[seq][1],ys[p]) ## start with zeros, add to each iteratively
            if s==1 or s==2 or s==4 or s==7: ## these are OFF states
#                 print('OFF "s" state '+str(s))
#                 print('"state" ' +str(state) + ' from ' + str(range((SFarr[2][seq][state][0]),(SFarr[2][seq][state][1]))))
#                 print('sums from '+str(range((SFarr[2][seq][state][0]),(SFarr[2][seq][state][1]))[0]) +
#                 ' - ' +str(range((SFarr[2][seq][state][0]),(SFarr[2][seq][state][1]))[-1]) + '\n')
                for p in range((SFarr[2][seq][state][0]),(SFarr[2][seq][state][1])): ##From 20-60 for example. SFarr[2] is the array of start to end pulses to sum
                    temp_OFF[seq] = np.add(temp_OFF[seq],ys[p]) ## start with zeros, add to each iteratively
#                     print(p)
    return temp_ON, temp_OFF, temp_ON_min, temp_ON_plu

In [ ]:
for i in range(len(ys_cut)):
#     print('#################### channel: ' + str(i) + ' ##########################')
    ON_sums[i], OFF_sums[i], ON_minus_sums[i], ON_plus_sums[i] = add_pulse(ys_cut[i], sequence)
    
# for i in range(len(ys_basesub)-12):
#     print('#################### channel: ' + str(i) + ' ##########################')
#     ON_sums[i], OFF_sums[i], ON_minus_sums[i], ON_plus_sums[i] = add_pulse(ys_basesub[i], sequence)

plt.legend() 
plt.show()

In [ ]:
end = time.time()
print('summing pulses into their states time: ' + str(end-start))  

In[14]:

In [ ]:
asym_ch = np.zeros((len(ON_sums), len(ON_sums[0][0])), dtype=np.float64) ## 1 Asym for each channel, not for each sequence (can change?)
asym_pm = np.zeros((len(ON_plus_sums),2, len(ON_plus_sums[0][0][0])), dtype=np.float64) ## 1 Asym for each channel, not for each sequence (can change?)

In [ ]:
def asym2(ON_arr, OFF_arr):
    tempasym = np.zeros((len(ON_arr[0])), dtype=np.float64)
    for seq in range(0, len(ON_arr)): ## number of sequences
#         print(seq)
        seqasym = ((ON_arr[seq]-OFF_arr[seq]) / (ON_arr[seq]+OFF_arr[seq]))
        tempasym = np.add(seqasym,tempasym)
    normedasym = tempasym/len(ON_arr)
    return normedasym

In [ ]:
for i in range(len(ON_sums)):
    asym_ch[i] = asym2(ON_sums[i], OFF_sums[i])
    asym_pm[i][0] = asym2(ON_plus_sums[i][0], ON_plus_sums[i][1])
    asym_pm[i][1] = asym2(ON_minus_sums[i][0], ON_minus_sums[i][1])
    # print('ch done')

In[13]:

In [ ]:
np.save(os.getcwd() + AsymSavename, asym_ch)
np.save(os.getcwd() + '/processed_data/'+ 'xs_cut', xs_cut)
np.save(os.getcwd() + pmSavename, asym_pm)
# np.save(os.getcwd() + AsymSavename, Asym)

In [ ]:
fullend = time.time()
print('full processing time: ' + str(fullend-fullstart))  
print('finished ' + str(datetime.now())) 
print('\n')

## end of data processing ##